<a href="https://colab.research.google.com/github/okana2ki/intro-to-AI/blob/main/Pose_Landmarker_for_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2024.4.29

動画ファイルに対してポーズランドマークを検出し、それを重畳した動画をファイルに保存するプログラム←GPT4を使用して作成

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mediapipe opencv-python-headless

In [ ]:
import cv2
import mediapipe as mp

# MediaPipeのポーズモジュールを初期化
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

# MediaPipeのポーズモジュールを初期化（複数人検出の設定）
# - model_complexity は、使用するポーズ検出モデルの複雑度を設定します。値が高いほど精度が向上しますが、
# 計算コストも増加し、処理速度が遅くなります。値は 0, 1, 2 から選べますが、これはモデルのバリエーションを
# 示すもので、検出可能な人数を指定するものではありません。
# - enable_segmentation は、背景セグメンテーションを有効にするかどうかを指定します。
# これが True に設定されていると、モデルが背景と人物の区分けを行い、結果として人物の検出精度が
# 向上することがあります。これもまた、検出される人数を増やす設定ではありません。
# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, enable_segmentation=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    # enable_segmentation=False,
    # min_detection_confidence=0.1,
    # min_detection_confidence はポーズが初めて検出される際の確信度のしきい値
    # min_tracking_confidence=0.1
    # min_tracking_confidence は、一度検出されたポーズの「トラッキング」を継続する際の確信度のしきい値

# 入力動画と出力動画のパス
input_video_path = '/content/drive/MyDrive/Colab_files/dance-sample.mp4'
input_audio_path = '/content/drive/MyDrive/Colab_files/dance-sample.mp3'
output_video_path = '/content/drive/MyDrive/Colab_files/annotated_video.mp4'
output_video_audio_path = '/content/drive/MyDrive/Colab_files/annotated_dance.mp4'

# dance-sample.mp4から音声を抽出し、dance-sample.mp3として保存します。ランドマーク重畳後の動画にこの音声を結合し、最終出力とします。
!ffmpeg -i /content/drive/MyDrive/Colab_files/dance-sample.mp4 -q:a 0 -map a /content/drive/MyDrive/Colab_files/dance-sample.mp3

# 動画ファイルを読み込み
cap = cv2.VideoCapture(input_video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 出力動画の設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# 動画の各フレームに対して処理
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR画像をRGBに変換
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # ポーズランドマークを描画
    mp_drawing = mp.solutions.drawing_utils
    annotated_image = image.copy()
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=results.pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))

    # RGB画像をBGRに戻す
    annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
    out.write(annotated_image)

# リソースの解放
cap.release()
out.release()
pose.close()

# ランドマークが描画された動画ファイルに音声ファイルを結合し、最終出力ファイルとします。
!ffmpeg -i /content/drive/MyDrive/Colab_files/annotated_video.mp4 -i /content/drive/MyDrive/Colab_files/dance-sample.mp3 -c:v copy -c:a aac -strict experimental /content/drive/MyDrive/Colab_files/annotated_dance.mp4

print("動画処理が完了しました。ファイルは以下に保存されています:", output_video_audio_path)
